In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv('./sign_mnist_train/sign_mnist_train.csv')
test_data = pd.read_csv('./sign_mnist_test/sign_mnist_test.csv')

x_train = train_data.iloc[:, 1:].values
x_test = test_data.iloc[:, 1:].values

y_train = train_data.iloc[:, 0].values #Integer labels from 0 - 25
y_test = test_data.iloc[:, 0].values

x_train = x_train / 255.0 #Normalizing the dataset
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout




In [4]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.5),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(26, activation='softmax')  # 26 output classes for the 26 hand signs
])

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=15, validation_data=(x_test, y_test))


Epoch 1/15


858/858 [==============================] - 13s 14ms/step - loss: 2.2248 - accuracy: 0.3144 - val_loss: 1.0459 - val_accuracy: 0.6672
Epoch 2/15
858/858 [==============================] - 11s 12ms/step - loss: 1.2915 - accuracy: 0.5744 - val_loss: 0.6381 - val_accuracy: 0.7585
Epoch 3/15
858/858 [==============================] - 11s 13ms/step - loss: 0.9840 - accuracy: 0.6704 - val_loss: 0.4565 - val_accuracy: 0.8401
Epoch 4/15
858/858 [==============================] - 11s 13ms/step - loss: 0.8111 - accuracy: 0.7282 - val_loss: 0.3745 - val_accuracy: 0.8829
Epoch 5/15
858/858 [==============================] - 11s 12ms/step - loss: 0.6930 - accuracy: 0.7666 - val_loss: 0.2509 - val_accuracy: 0.9414
Epoch 6/15
858/858 [==============================] - 11s 13ms/step - loss: 0.6070 - accuracy: 0.7952 - val_loss: 0.2440 - val_accuracy: 0.9141
Epoch 7/15
858/858 [==============================] - 11s 13ms/step - loss: 0.5453 - accuracy: 0.8155 - val_loss: 0.1792 - val_accura

In [7]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


225/225 [==============================] - 1s 2ms/step - loss: 0.0828 - accuracy: 0.9789
Test accuracy: 0.9789459109306335


In [8]:
#Run this to save the model
model.save('hand_sign_mnist_model.h5')


c:\Users\karte\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('hand_sign_mnist_model.h5')


In [10]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2)
mp_draw = mp.solutions.drawing_utils

# Start webcam feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the image to RGB as Mediapipe uses RGB images
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect hands in the frame
    results = hands.process(rgb_frame)
    
    # Draw landmarks and connections if hands are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:

            # Get image dimensions
            h, w, c = frame.shape

            # Get coordinates of hand landmarks
            landmark_points = []
            for lm in hand_landmarks.landmark:
                cx, cy = int(lm.x * w), int(lm.y * h)
                landmark_points.append((cx, cy))

             # Calculate the bounding box around the hand
            x_min = min([point[0] for point in landmark_points])
            x_max = max([point[0] for point in landmark_points])
            y_min = min([point[1] for point in landmark_points])
            y_max = max([point[1] for point in landmark_points])

            # Add some padding to the bounding box
            padding = 20
            x_min = max(0, x_min - padding)
            x_max = min(w, x_max + padding)
            y_min = max(0, y_min - padding)
            y_max = min(h, y_max + padding)

            # Calculate the width and height of the bounding box
            box_width = x_max - x_min
            box_height = y_max - y_min

            # Find the larger of the width or height to make the bounding box a square
            box_size = max(box_width, box_height)

            # Adjust the x and y coordinates to make the bounding box square
            x_center = (x_min + x_max) // 2
            y_center = (y_min + y_max) // 2

            x_min = max(0, x_center - box_size // 2)
            x_max = min(w, x_center + box_size // 2)
            y_min = max(0, y_center - box_size // 2)
            y_max = min(h, y_center + box_size // 2)

            # Extract the region of interest (hand) as a square
            hand_roi = frame[y_min:y_max, x_min:x_max]


            # Optionally resize the ROI to match CNN input size
            
            hand_roi_resized = cv2.resize(hand_roi, (28, 28))
            hand_roi_resized = cv2.cvtColor(hand_roi_resized, cv2.COLOR_BGR2GRAY)
            #hand_roi_resized = cv2.GaussianBlur(hand_roi_resized, (5, 5), 0)
            hand_cnn_input = hand_roi_resized / 255.0
            hand_cnn_input = np.expand_dims(hand_cnn_input, axis=-1)
            hand_cnn_input = np.expand_dims(hand_cnn_input, axis=0)
            predictions = model.predict(hand_cnn_input)
            predicted_class = np.argmax(predictions, axis=1)[0]
            cv2.putText(frame, chr(predicted_class + 65), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                1, (0, 255, 0), 2, cv2.LINE_AA)
            #print(predictions)
            

            # Draw the bounding box around the hand
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        
    # Display the result
    cv2.imshow("Hand Detection", frame)
    
    # Break loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 97ms/step


c:\Users\karte\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 [==============================] - 0s 22ms/step
